# GCN
Build the model with the following structure using the PyTorch Geometric library:
1. Input layer
2. 1st hidden layer: GCNConv + ReLU
3. 2nd hidden layer: GCNConv + ReLU
4. 3rd hidden layer: GCNConv + global mean pooling + Dropout
5. Output layer: Linear
* Loss function: CrossEntropyLoss
* Optimizer: Adam

**WARNING** <br>
Execute only colab


In [ ]:
# install torch-geometric and related libraries
!pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
!pip install torch-geometric
!pip install scipy==1.8.0

## Load the dataset

In [1]:
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

# load dataset
dataset = TUDataset(root='/tmp/MUTAG', name='MUTAG')

# split dataset into train, validation and test
dataset = dataset.shuffle()
train_dataset = dataset[:140]
val_dataset = dataset[140:]

# create dataloader
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

/Users/takamatsunaoki/opt/anaconda3/envs/ai-master/lib/python3.7/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


## Build the model

In [ ]:
# define model
# 1. Input layer
# 2. 1st hidden layer: GCNConv + ReLU
# 3. 2nd hidden layer: GCNConv + ReLU
# 4. 3rd hidden layer: GCNConv + global mean pooling + Dropout
# 5. Output layer: Linear

import torch
from torch.nn import Linear, ReLU, Dropout
from torch_geometric.nn import GCNConv, global_mean_pool

n_h = 64

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, n_h)
        self.conv2 = GCNConv(n_h, n_h)
        self.conv3 = GCNConv(n_h, n_h)
        self.fc = Linear(n_h, dataset.num_classes)

        self.relu = ReLU()
        self.dropout = Dropout(p=0.5)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, batch_size
        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.conv3(x, edge_index)
        x = self.relu(x)
        x = global_mean_pool(x, batch)
        x = self.dropout(x)
        x = self.fc(x)
        return x

net = Net()
net.cuda() # use GPU

# Train the model

In [ ]:
# define loss function and optimizer
# * Loss function: CrossEntropyLoss
# * Optimizer: Adam
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

criterion = CrossEntropyLoss()
optimizer = Adam(net.parameters(), lr=0.01)

# define evaluation function
def evaluate(loader):
    correct = 0
    for data in loader:
        data = data.cuda() # use GPU
        out = net(data)
        pred = out.argmax(dim=1)
        correct += int((pred == data.y).sum())
    return correct / len(loader.dataset)  * 100

# train model
for epoch in range(200):
    net.train()
    for data in train_loader:
        data = data.cuda() # use GPU
        optimizer.zero_grad()
        out = net(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()

    # evaluate model
    net.eval()
    train_acc = evaluate(train_loader)
    val_acc = evaluate(val_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}')

## Evaluate the model

In [ ]:
net.eval()
test_acc = evaluate(val_loader)
print(f'Test Acc: {test_acc:.4f}')